In [2]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import torch

# Assignment 3

## Image Classification Using RF and SVM

1. Resize the train/test images to 64 × 64 and compute HoG features using cells of 8×8 pixels, blocks of 4×4 cells and 4 bins. This should yeild a feature vector of size 1600 per image.
2. Fit a non-linear SVM classifier (use RBF kernel with gamma=‘auto’ and C=1) on the features and the class labels of the training images. 
3. Predict labels of the test images by feeding the test features to the trained classifier and calculate classification accuracy.
4. Tune values of hyperparameters ‘gamma’ and ‘C’ to achieve test accuracy greater than 25%.
5. Fit a Random Forest(RF) classifier (set n estimators=10, max depth=5 and criterion=‘entropy’) on the features and the class labels of the training images.
6. Predict labels of the test images by feeding the test features to the trained classifier and calculate classification accuracy.
7. Tune values of hyperparameters ‘n estimators’ and ‘max depth’ to achieve test accuracy greater than 25%. 
8. Compare results of SVM and RF classifiers. Which one provides better results? Experiment training both classifiers with a range of random stats and measure classification accuracy of the test set. Which classifier is more stable or robust to the change in random state? 

In [3]:
# load in the dataset
flower_subset = np.load("flower_subset.npz")
for file in flower_subset.files:
    print(file)
    print(type(flower_subset[file]))
    print(flower_subset[file])

train_images
<class 'numpy.ndarray'>
[[[ 49  48  46 ...  65  56  50]
  [ 42  41  39 ... 110 111 111]
  [115 114 113 ... 167 166 165]
  ...
  [ 91  88  87 ...  47  52  61]
  [ 69  73  74 ...  81  78  72]
  [ 65  63  65 ... 178 179 178]]

 [[ 64  62  59 ...  76  73  72]
  [ 68  68  69 ...  81  80  79]
  [ 80  81  83 ...  72  72  72]
  ...
  [ 70  70  69 ...  98  98  98]
  [ 94  95  96 ...  83  82  81]
  [ 87  90  89 ...  91  91  92]]

 [[ 10  10  11 ...  87  87  87]
  [ 86  86  85 ... 184 184 184]
  [184 184 184 ... 205 205 206]
  ...
  [ 78  83  86 ...  54  54  55]
  [ 55  55  56 ...  22  21  21]
  [ 21  21  21 ...  19  21  22]]

 ...

 [[ 18  18  18 ...  39  43  45]
  [ 46  45  44 ...  36  33  30]
  [ 52  53  45 ... 153 151 147]
  ...
  [ 22  23  23 ...  31  31  32]
  [ 29  31  34 ... 122 115  95]
  [ 82 104  94 ...  36  37  38]]

 [[ 40  40  42 ...  69  61  54]
  [ 46  42  37 ...  36  29  24]
  [ 20  19  17 ...  38  39  39]
  ...
  [ 17  17  18 ...  42  41  39]
  [ 37  35  33 ...  64 

In [4]:
# assign meaningful names to each set in the dataset
train_images = flower_subset["train_images"]
train_labels = flower_subset["train_labels"]
test_images = flower_subset["test_images"]
test_labels = flower_subset["test_labels"]

In [6]:
def display_single_image(img, name):
    """Display a single image nicely."""
    plt.figure(figsize=(20, 20))
    plt.subplot(111), plt.imshow(img, cmap="gray")
    plt.title(name), plt.xticks([]), plt.yticks([])
    plt.show()

In [15]:
def half_resolution(img):
    return cv2.resize(img, dsize=(0,0), fx=0.5, fy=0.5)

In [16]:
# resize to 64x64
train_images = np.array(list(map(half_resolution, train_images)))
test_images = np.array(list(map(half_resolution, test_images)))

In [ ]:
# create HoG Object
# winSize is the size of the image cropped to multiple of the cell size
# all arguments should be given in terms of number of pixels
hog = cv2.HOGDescriptor(_winSize=(img_size[1] // cell_size[1] * cell_size[1],
                                  img_size[0] // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)